# computation of shape metrics: parallelization

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math, random, time, os, sys, string
import geopandas as gpd
from shapely.geometry import Polygon, Point, MultiPolygon, LineString

In [2]:
shp_path = '/Users/rodrigo/Library/Mobile Documents/com~apple~CloudDocs/Documents/tfg/data/ups/ramdom_pf.shp'
shpfile = gpd.read_file(shp_path)

In [ ]:
"python setup.py build_ext --inplace"

In [16]:
import shapemetrics

def shapemetrics(polygon):
    results = []
    featPntLst = shapemetrics.ConvertToGridPnts(polygon)
    centroid = polygon.centroid
    Xc, Yc = centroid.x, centroid.y
    A = polygon.area
    r = (A / math.pi)**.5       # radius of equal area circle (circle with area equal to shape area)

    # COHESION #
    interdist = shapemetrics.interpointDistance(featPntLst)
    circ_interD = r * .9054
    CohesionIndex = circ_interD / interdist
    
    # PROXIMITY #
    D_to_Center, EAC_pix = shapemetrics.proximity(featPntLst,Xc,Yc,r)    
    # avg distance to center for equal area circle...
    circD = r * (2.0/3.0)
    # proximity index (circle / shape)...
    ProximityIndex = circD / D_to_Center         

    # SPIN #
    shpMOI = Helper_tool.spin(featPntLst,Xc,Yc)
    # moment of inertia for equal area circle...
    circ_MOI = .5 * r**2
    # calculate spin index (circle / shape)...
    Spin = circ_MOI / shpMOI    
    
    return '???'

In [ ]:
from joblib import Parallel, delayed
num_cores = 6
metrics = Parallel(n_jobs = num_cores, verbose = 1)(delayed(shapemetrics)(pol) for pol in shapefile['geometry'])

In [ ]:
metrics = pd.DataFrame(results, columns=['area', 'interdist', 'cohesion'])
shpfilemetrics = pd.concat([shpfile, metrics], axis = 1)
shpfilemetrics.to_file('/Users/rodrigo/Documents/tfg/data/ups/shapemetrics.shp')

In [ ]:
# I need to do it for the actual shapes AND for the potential shapes!
# Cythonize spin and proximity